In [1]:
import numpy as np
import pandas as pd
import src.helpers as hlp
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from collections import defaultdict

from torch import nn, optim
from keras_preprocessing.sequence import pad_sequences
from torch.utils.data import Dataset, DataLoader

import transformers
from transformers import XLNetTokenizer, XLNetModel, AdamW, XLNetForSequenceClassification
import torch
import contractions


ModuleNotFoundError: No module named 'transformers'

In [5]:
t_pos = pd.read_table('/kaggle/input/twitter-full/train_pos_full.txt', header=None, names=['tweet'], dtype=str,on_bad_lines='skip')
t_pos['label'] = 1
t_neg = pd.read_table('/kaggle/input/twitter-full/train_neg_full.txt', header=None, names=['tweet'], dtype=str,on_bad_lines='skip')
t_neg['label'] = 0
df = pd.concat((t_pos,t_neg))

In [6]:
#df['tweet'] = df['tweet'].apply(lambda x: hlp.remove_stopwords(x))
df['tweet'] = df['tweet'].apply(lambda x: hlp.remove_punct(x))
df['tweet'] = df['tweet'].apply(lambda x: hlp.add_space(x))
df['tweet'] = df['tweet'].apply(lambda x: hlp.remove_white_space(x))
#df['tweet'] = df['tweet'].apply(lambda x: hlp.remove_words_digits(x))
df['tweet'] = df['tweet'].apply(lambda x: hlp.to_lower(x))
df['tweet'] = df['tweet'].apply(lambda x: hlp.unslang(x))
df['tweet'] = df['tweet'].apply(lambda x: contractions.fix(x))
#df['tweet'] = df['tweet'].apply(lambda x: hlp.remove_specific_words(x))
#df['tweet'] = df['tweet'].apply(lambda x: hlp.remove_single_char(x))
df['tweet'] = df['tweet'].apply(lambda x: hlp.lemmatize(x))
df = df[df['tweet'] != '']
df = df.drop_duplicates()
df.reset_index(inplace=True)

In [7]:
MAX_LEN = 256

In [8]:
class CustomDataset(Dataset):

    def __init__(self, tweets, labels, tokenizer, max_len):
        self.tweets = tweets
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.tweets)
    
    def __getitem__(self, index):
        tweet = str(self.tweets[index])
        label = self.labels[index]

        encoding = self.tokenizer.encode_plus(
        tweet,
        add_special_tokens=True,
        max_length=self.max_len,
        return_token_type_ids=False,
        pad_to_max_length=False,
        return_attention_mask=True,
        return_tensors='pt',
        )

        input_ids = pad_sequences(encoding['input_ids'], maxlen=MAX_LEN, dtype=torch.Tensor ,truncating="post",padding="post")
        input_ids = input_ids.astype(dtype = 'int64')
        input_ids = torch.tensor(input_ids) 

        attention_mask = pad_sequences(encoding['attention_mask'], maxlen=MAX_LEN, dtype=torch.Tensor ,truncating="post",padding="post")
        attention_mask = attention_mask.astype(dtype = 'int64')
        attention_mask = torch.tensor(attention_mask)       

        return {
        'review_text': tweet,
        'input_ids': input_ids,
        'attention_mask': attention_mask.flatten(),
        'targets': torch.tensor(label, dtype=torch.long)
        }

In [9]:
X_train, X_test, y_train, y_test = train_test_split (df['tweet'], df['label'] , test_size=0.2)

In [10]:
xlnet_tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

In [11]:
tweet_train_dataset = CustomDataset(tweets=X_train.to_numpy(), labels=y_train.to_numpy(), tokenizer=xlnet_tokenizer, max_len=MAX_LEN)
tweet_test_dataset = CustomDataset(tweets=X_test.to_numpy(), labels=y_test.to_numpy(), tokenizer=xlnet_tokenizer, max_len=MAX_LEN)


In [12]:
BATCH_SIZE = 32

In [13]:
tweet_train_loader = DataLoader(tweet_train_dataset, batch_size=BATCH_SIZE)
tweet_test_loader = DataLoader(tweet_test_dataset, batch_size=BATCH_SIZE)


In [14]:
EPOCHS = 3

model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels = 2)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'logits_proj.weight', 'logits_proj.bias', 'sequence_summary.summary.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)

In [16]:
from sklearn import metrics
def train_epoch(model, data_loader, optimizer, device, n_examples):
    model = model.train()
    losses = []
    acc = 0
    counter = 0
  
    for d in data_loader:
        input_ids_shape = d["input_ids"].shape
        input_ids = d["input_ids"].reshape(input_ids_shape[0],input_ids_shape[2]).to(device)
        attention_mask = d["attention_mask"].to(device)
        targets = d["targets"].to(device)
        
        outputs = model(input_ids=input_ids, token_type_ids=None, attention_mask=attention_mask, labels = targets)
        loss = outputs[0]
        logits = outputs[1]

        _, prediction = torch.max(outputs[1], dim=1)
        targets = targets.cpu().detach().numpy()
        prediction = prediction.cpu().detach().numpy()
        accuracy = metrics.accuracy_score(targets, prediction)

        acc += accuracy
        losses.append(loss.item())
        
        loss.backward()

        optimizer.step()
        optimizer.zero_grad()
        if counter % 1000 == 0:
            save_str = '/kaggle/working/xlnet_model_'+str(counter)+'.bin'
            torch.save(model.state_dict(), save_str)
        if counter%100 == 0:
                print("{}th iteration over {} = {}%".format(counter, len(data_loader), 100.0*counter/len(data_loader)))
                print("running mean_acc : ", acc/counter if counter != 0 else acc)
                print("running mean_loss : ", np.mean(losses))
        counter = counter + 1
        
        

    return acc / counter, np.mean(losses)

In [17]:
def eval_model(model, data_loader, device, n_examples):
    model = model.eval()
    losses = []
    acc = 0
    counter = 0
  
    with torch.no_grad():
        for d in data_loader:
            input_ids_shape = d["input_ids"].shape
            input_ids = d["input_ids"].reshape(input_ids_shape[0],input_ids_shape[2]).to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["targets"].to(device)
            
            outputs = model(input_ids=input_ids, token_type_ids=None, attention_mask=attention_mask, labels = targets)

            loss = outputs[0]
            logits = outputs[1]

            _, prediction = torch.max(outputs[1], dim=1)
            targets = targets.cpu().detach().numpy()
            prediction = prediction.cpu().detach().numpy()
            accuracy = metrics.accuracy_score(targets, prediction)

            acc += accuracy
            losses.append(loss.item())
            if counter%100 == 0:
                print("{}th iteration over {} = {}%".format(counter, len(data_loader), 100.0*counter/len(data_loader)))
                print("running mean_acc : ", acc/counter if counter != 0 else acc)
                print("running mean_loss : ", np.mean(losses))
            counter += 1
            

    return acc / counter, np.mean(losses)

In [18]:
import logging 
logging.basicConfig(level = logging.ERROR)
history = defaultdict(list)
best_accuracy = 0

for epoch in range(EPOCHS):
    print(f'Epoch {epoch + 1}/{EPOCHS}')
    print('-' * 10)

    train_acc, train_loss = train_epoch(
        model,
        tweet_train_loader,     
        optimizer, 
        device, 
        len(X_train)
    )

    print(f'Train loss {train_loss} Train accuracy {train_acc}')

    val_acc, val_loss = eval_model(
        model,
        tweet_test_loader, 
        device, 
        len(X_test)
    )

    print(f'Val loss {val_loss} Val accuracy {val_acc}')
    print()

    history['train_acc'].append(train_acc)
    history['train_loss'].append(train_loss)
    history['val_acc'].append(val_acc)
    history['val_loss'].append(val_loss)

    if val_acc > best_accuracy:
        torch.save(model.state_dict(), '/kaggle/working/xlnet_best_model.bin')
        best_accuracy = val_acc

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Epoch 1/3
----------


0th iteration over 55193 = 0.0%
running mean_acc :  0.46875
running mean_loss :  0.7471943497657776


100th iteration over 55193 = 0.1811823963183737%
running mean_acc :  0.619375
running mean_loss :  0.638375850302158


200th iteration over 55193 = 0.3623647926367474%
running mean_acc :  0.68953125
running mean_loss :  0.5699611077112938


...

16600th iteration over 55193 = 30.076277788850035%
running mean_acc :  0.8382981927710843
running mean_loss :  0.3582678530130729


16700th iteration over 55193 = 30.25746018516841%
running mean_acc :  0.8384113023952096
running mean_loss :  0.3580475865359878


16800th iteration over 55193 = 30.438642581486782%
running mean_acc :  0.8385342261904762
running mean_loss :  0.3578274710803761


16900th iteration over 55193 = 30.619824977805155%
running mean_acc :  0.8386020710059171
running mean_loss :  0.35763847808514565


17000th iteration over 55193 = 30.80100737412353%
running mean_acc :  0.8387334558823529
running mean_loss :  0.35737619518096025
